### Библиотеки / данные

импортируем библиотеки numpy и pandas

In [2]:
import numpy as np
import pandas as pd

задаем некоторые настройки pandas, регулирующие формат вывода

In [11]:
pd.options.display.max_rows = 10

загружаем данные

In [12]:
tips = pd.read_csv("../data/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [13]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [14]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


### Механизм GroupBy

<img src = '../images/split_apply_combine.png' style='width: 600px;'/>

In [15]:
df = pd.DataFrame({'x': ['a','a','b','b','c','c'],
                   'y': [2,4,0,5,5,10]})
df

,x,y
0,a,2
1,a,4
2,b,0
3,b,5
4,c,5
5,c,10


In [16]:
groups = df.groupby(['x'])
groups

получаем информацию о количестве групп, которые будут созданы

In [17]:
groups.ngroups

3

получаем информацию о количестве элементов в каждой группе

In [18]:
groups.size()

x
a    2
b    2
c    2
dtype: int64

что представляют из себя найденные группы?

In [19]:
groups.groups

{'a': Int64Index([0, 1], dtype='int64'),
 'b': Int64Index([2, 3], dtype='int64'),
 'c': Int64Index([4, 5], dtype='int64')}

получаем данные конкретной группы

In [20]:
groups.get_group('b')

,x,y
2,b,0
3,b,5


извлекаем первую строку каждой группы

In [21]:
groups.nth([1])

,y
x,
a,4
b,5
c,10


обход групп:

In [26]:
for key, group in groups:
    print(key)
    print(group) 

a
   x  y
0  a  2
1  a  4
b
   x  y
2  b  0
3  b  5
c
   x   y
4  c   5
5  c  10


вычисление среднего

In [27]:
groups.y.mean()

x
a    3.0
b    2.5
c    7.5
Name: y, dtype: float64

### Типы группировок

#### по столбцам: 

In [28]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [29]:
tips.groupby(['day','time']).tip.mean().unstack()

time,Dinner,Lunch
day,,
Fri,2.940000,2.382857
Sat,2.993103,NaN
Sun,3.255132,NaN
Thur,3.000000,2.767705


#### по уровням индекса

создаем копию данных и заново индексируем ее

In [30]:
copy_tips = tips.copy()
copy_tips = copy_tips.set_index(['day', 'time'])
copy_tips

total_bill   tip smoker  size   tip_pct
day  time                                           
Sun  Dinner       16.99  1.01     No     2  0.059447
     Dinner       10.34  1.66     No     3  0.160542
     Dinner       21.01  3.50     No     3  0.166587
     Dinner       23.68  3.31     No     2  0.139780
     Dinner       24.59  3.61     No     4  0.146808
...                 ...   ...    ...   ...       ...
Sat  Dinner       29.03  5.92     No     3  0.203927
     Dinner       27.18  2.00    Yes     2  0.073584
     Dinner       22.67  2.00    Yes     2  0.088222
     Dinner       17.82  1.75     No     2  0.098204
Thur Dinner       18.78  3.00     No     2  0.159744

[244 rows x 5 columns]

группировать можем по одному или нескольким уровням индекса, передавая соответствующие значения столбцов аргументу level

In [31]:
copy_tips.groupby(level=['time']).sum()

,total_bill,tip,size,tip_pct
time,,,,
Dinner,3660.30,546.07,463,28.075131
Lunch,1167.47,185.51,164,11.160699


In [32]:
copy_tips.groupby(level = ['day', 'time']).mean()

total_bill       tip      size   tip_pct
day  time                                            
Fri  Dinner   19.663333  2.940000  2.166667  0.158916
     Lunch    12.845714  2.382857  2.000000  0.188765
Sat  Dinner   20.441379  2.993103  2.517241  0.153152
Sun  Dinner   21.410000  3.255132  2.842105  0.166897
Thur Dinner   18.780000  3.000000  2.000000  0.159744
     Lunch    17.664754  2.767705  2.459016  0.161301

#### с использованием функции 

In [33]:
copy_tips.head()

total_bill   tip smoker  size   tip_pct
day time                                           
Sun Dinner       16.99  1.01     No     2  0.059447
    Dinner       10.34  1.66     No     3  0.160542
    Dinner       21.01  3.50     No     3  0.166587
    Dinner       23.68  3.31     No     2  0.139780
    Dinner       24.59  3.61     No     4  0.146808

In [34]:
copy_tips.groupby(len, level='day').count()

,total_bill,tip,smoker,size,tip_pct
day,,,,,
3,182,182,182,182,182
4,62,62,62,62,62


In [35]:
tips.day.value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

#### с использованием массива 

массив случайных меток групп:

In [ ]:
np.random.seed(123)
rnd_array = np.random.choice(['first_group', 'second_group'], 
                             size=tips.shape[0],
                             p = [0.4, 0.6])
rnd_array[0:5]

передаем массив меток групп в метод groupby

In [ ]:
tips.groupby(rnd_array).count()

In [ ]:
93/(93+151)

#### бонус

тип данных:

In [ ]:
tips.dtypes

In [ ]:
group_tips = tips.groupby(tips.dtypes, axis=1)

In [ ]:
group_tips.size()

### Агрегирование групп

создаем объект groupby:

In [ ]:
group_tips = tips.groupby(['day', 'smoker'])['tip_pct']

проверка эквивалентности методов:

In [ ]:
group_tips.agg == group_tips.aggregate

собственная функция:

In [ ]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

group_tips.agg([peak_to_peak])

комбинирование:

In [ ]:
group_tips.agg([peak_to_peak, 'mean'])

задаем имена столбцов:

In [ ]:
group_tips.agg([('delta_max_min', peak_to_peak), ('mean_value','mean')])

отдельные наборы функций для каждого столбца:

In [ ]:
group_tips = tips.groupby(['day', 'smoker'])['tip_pct', 'total_bill']

In [ ]:
group_tips.agg({'tip_pct':[('max_value', np.max),
                           ('min_value','min')],
                'total_bill': 'sum'})

### Преобразование групп

#### метод transform

<img src = '../images/transform.png' style='width: 900px;'/>

In [ ]:
df = pd.DataFrame({'Col1': ['A', 'B', 'C', 'C', 'B', 'B', 'A'],
                   'Col2': [1, 2, 3, 4, 2, 5, 3]})
df

In [ ]:
df['Col3'] = df.groupby('Col1').transform(sum)
df.sort_values('Col1')

объект groupby:

In [ ]:
group_tips = tips.groupby(['smoker'])['total_bill']

нормирование:

In [ ]:
norm = lambda x: (x - x.mean())/x.std()
group_tips.transform(norm)

In [ ]:
tips_copy = tips.copy()
tips_copy['total_bill_norm'] = group_tips.transform(norm)
tips_copy.head()

In [ ]:
tips_copy.groupby('smoker').total_bill_norm.agg(['mean', 'std'])

#### метод apply

функция:

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

применяем метод apply

In [ ]:
tips.groupby('smoker').apply(top)

### Исключение групп

создаем данные для наших примеров

In [9]:
df = pd.DataFrame({'Label': list('AABCCC'),
                   'Values': [1, 2, 3, 4, np.nan, 8]})
df

,Label,Values
0,A,1.0
1,A,2.0
2,B,3.0
3,C,4.0
4,C,NaN
5,C,8.0


удаляем группы с менее чем двумя возможными значениеми

In [11]:
f = lambda x: x.Values.count() > 1
type (df.groupby('Label').filter(f))

pandas.core.frame.DataFrame

удаляем группы, в которых есть пропуски

In [5]:
f = lambda x: x.Values.isnull().sum() == 0
df.groupby('Label').filter(f)

,Label,Values
0,A,1.0
1,A,2.0
2,B,3.0


### Сводная таблица

средние

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

вывод по строкам и столбцам

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

включение частичных итогов:

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

задаем функцию:

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=max, margins=True)